我们经常遇到这样的推断数据：

$$\mathcal{D} = \{(\boldsymbol{x_i},y_i)|i=1,\ldots,n, \boldsymbol{x_i} \in \mathcal{X}, y_i \in \mathbb{R}\}$$

其中 $\boldsymbol{x_i}$ 是输入，而 $y_i$ 是目标输出，我们希望预测新的输入 $\boldsymbol{x_*}$ 的输出 $y_*$。

从贝叶斯的角度来看，我们可以构建一个模型来定义所有可能函数的分布$(f: \mathcal{X} \rightarrow \mathbb{R})$。我们可以将初始信念编码为这些函数的先验。基于观测数据和贝叶斯定理来推断后验分布。然后基于后验得到输入 $\boldsymbol{x_*}$ 的对 $y_*$ 的预测分布。

高斯过程 Gaussian processes (GPs) 是一种函数的概率分布。对于这种分布，我们可以进行可行的推断。    
它们可以被视为高斯概率分布到函数空间的推广。即多元高斯分布定义了有限随机变量集的分布，高斯过程定义了无限随机变量集（例如实数）的分布。GP域不限于实数，任何具有点积的空间都适用。    
与多元高斯分布类似，GP 由其均值 $\mu$ 和协方差 $k$ 定义。但是对于 GP，这些是函数，而不是向量，即 $\mu: \mathcal{X} \rightarrow \mathbb{R}$ 和 $k: \mathcal{X} \times \mathcal{X} \rightarrow \mathbb{R}$。      
为方便又不是一般性，下面说明中我们假设均值为零，即 $\mu(\boldsymbol{x}) = 0$。

![](samples_from_prior.png)

![](samples_from_posterior.png)


上图来自具有相同均值和协方差函数的两个高斯过程的样本。这里有 $\mathcal{X} = \mathbb{R}$。先验样本来自没有任何数据的高斯过程，后验样本取自有观测数据高斯过程，其中数据显示为黑色方块。黑色虚线代表过程的平均值，灰色阴影区域覆盖每个输入的标准偏差的两倍。彩色线条是来自过程的样本。

生成样本的代码如下：

```python
from numpy.random import seed
from infpy.gp import GaussianProcess, gp_1D_X_range, gp_plot_samples_from
from pylab import plot, savefig, title, close, figure, xlabel, ylabel

# seed RNG to make reproducible and close all existing plot windows
seed(2)
close('all')

#
# Kernel
#
from infpy.gp import SquaredExponentialKernel as SE
kernel = SE([1])

#
# Part of X-space we will plot samples from
#
support = gp_1D_X_range(-10.0, 10.01, .125)

#
# Plot samples from prior
#
figure()
gp = GaussianProcess([], [], kernel)
gp_plot_samples_from(gp, support, num_samples=3)
xlabel('x')
ylabel('f(x)')
title('Samples from the prior')
savefig('samples_from_prior.png')
savefig('samples_from_prior.eps')

#
# Data
#
X = [[-5.], [-2.], [3.], [3.5]]
Y = [2.5, 2, -.5, 0.]

#
# Plot samples from posterior
#
figure()
plot([x[0] for x in X], Y, 'ks')
gp = GaussianProcess(X, Y, kernel)
gp_plot_samples_from(gp, support, num_samples=3)
xlabel('x')
ylabel('f(x)')
title('Samples from the posterior')
savefig('samples_from_posterior.png')
savefig('samples_from_posterior.eps')
```

## 协方差函数 $k$

假设均值函数为0，则 GP 由观测数据 $\mathcal{D}$ 和 协方差函数 $k$ 决定。观测数据是确定的，因此我们只需要定义协方差函数。    
幸运的是，我们有一个庞大的可能协方差函数库，每个协方差函数代表函数空间上的不同先验。    

![](covariance_function_se.png)

![](covariance_function_matern_52.png)

![](covariance_function_periodic.png)

从具有相同数据和不同协方差函数的 GP 中抽样。不同协方差函数的 GP 后验的样本具有不同的特征。periodic 协方差函数的主要特征是具有周期性。其他协方差函数以不同的方式影响样本的平滑度。

产生样本的代码如下：

```python
from numpy.random import seed
from infpy.gp import GaussianProcess, gp_1D_X_range, gp_plot_samples_from
from pylab import plot, savefig, title, close, figure, xlabel, ylabel
from infpy.gp import SquaredExponentialKernel as SE
from infpy.gp import Matern52Kernel as Matern52
from infpy.gp import Matern52Kernel as Matern32
from infpy.gp import RationalQuadraticKernel as RQ
from infpy.gp import NeuralNetworkKernel as NN
from infpy.gp import FixedPeriod1DKernel as Periodic
from infpy.gp import noise_kernel as noise

# seed RNG to make reproducible and close all existing plot windows
seed(2)
close('all')

#
# Part of X-space we will plot samples from
#
support = gp_1D_X_range(-10.0, 10.01, .125)

#
# Data
#
X = [[-5.], [-2.], [3.], [3.5]]
Y = [2.5, 2, -.5, 0.]

def plot_for_kernel(kernel, fig_title, filename):
  figure()
  plot([x[0] for x in X], Y, 'ks')
  gp = GaussianProcess(X, Y, kernel)
  gp_plot_samples_from(gp, support, num_samples=3)
  xlabel('x')
  ylabel('f(x)')
  title(fig_title)
  savefig('%s.png' % filename)
  savefig('%s.eps' % filename)
  
plot_for_kernel(
  kernel=Periodic(6.2),
  fig_title='Periodic',
  filename='covariance_function_periodic'
)

plot_for_kernel(
  kernel=RQ(1., dimensions=1),
  fig_title='Rational quadratic',
  filename='covariance_function_rq'
)

plot_for_kernel(
  kernel=SE([1]),
  fig_title='Squared exponential',
  filename='covariance_function_se'
)

plot_for_kernel(
  kernel=SE([3.]),
  fig_title='Squared exponential (long length scale)',
  filename='covariance_function_se_long_length'
)

plot_for_kernel(
  kernel=Matern52([1.]),
  fig_title='Matern52',
  filename='covariance_function_matern_52'
)

plot_for_kernel(
  kernel=Matern32([1.]),
  fig_title='Matern32',
  filename='covariance_function_matern_32'
)
```

## 协方差函数和噪音数据的结合

此外，协方差函数的逐点乘积和本身就是协方差函数。通过这种方式，我们可以组合简单的协方差函数来表示我们对函数的更复杂的信念。    
通常我们正在对一个我们实际上无法获取的目标值 $y$ 建模，我们只有它的有噪音版本 $y+\epsilon$。    
如果我们假设 $\epsilon$ 是一个方差为 $\sigma_n^2$ 的高斯分布，我们就能将噪音合并进协方差函数中。    
我们的带噪声的高斯过程（noisy GP）的协方差函数 $k_{\textrm{noise}}(x_1,x_2)$ 需要能够识别输入 $x_1$ 和 $x_2$ 是否相同，因为我们可能在 $\mathcal{X}$ 的同一点有两个带噪声的测量值。
$$k_{\textrm{noise}}(x_1,x_2) = k(x_1,x_2) + \delta(x_1=x_2) \sigma_n^2\$$

![](noise_low.png)
![](noise_mid.png)
![](noise_high.png)

生成上图的代码：
    
```python
from numpy.random import seed
from infpy.gp import GaussianProcess, gp_1D_X_range, gp_plot_prediction
from pylab import plot, savefig, title, close, figure, xlabel, ylabel
from infpy.gp import SquaredExponentialKernel as SE
from infpy.gp import noise_kernel as noise

# close all existing plot windows
close('all')

#
# Part of X-space we are interested in
#
support = gp_1D_X_range(-10.0, 10.01, .125)

#
# Data
#
X = [[-5.], [-2.], [3.], [3.5]]
Y = [2.5, 2, -.5, 0.]

def plot_for_kernel(kernel, fig_title, filename):
  figure()
  plot([x[0] for x in X], Y, 'ks')
  gp = GaussianProcess(X, Y, kernel)
  mean, sigma, LL = gp.predict(support)
  gp_plot_prediction(support, mean, sigma)
  xlabel('x')
  ylabel('f(x)')
  title(fig_title)
  savefig('%s.png' % filename)
  savefig('%s.eps' % filename)
  
plot_for_kernel(
  kernel=SE([1.]) + noise(.1),
  fig_title='k = SE + noise(.1)',
  filename='noise_mid'
)

plot_for_kernel(
  kernel=SE([1.]) + noise(1.),
  fig_title='k = SE + noise(1)',
  filename='noise_high'
)

plot_for_kernel(
  kernel=SE([1.]) + noise(.0001),
  fig_title='k = SE + noise(.0001)',
  filename='noise_low'
)
```

## 学习协方差函数参数

大多数常用的协方差函数都是参数化的。如果我们对问题的理解有信心，则可以使用固定参数。或者我们可以将它们视为贝叶斯推理任务中的超参数，并通过最大似然估计或共轭梯度下降等技术对其进行优化。

![](learning_first_guess.png)
![](learning_learnt.png)


我们看到第二张图中的预测似乎更准确地拟合数据，这是学习超参数的效果。

生成上图的代码：

```python
from numpy.random import seed
from infpy.gp import GaussianProcess, gp_1D_X_range
from infpy.gp import gp_plot_prediction, gp_learn_hyperparameters
from pylab import plot, savefig, title, close, figure, xlabel, ylabel
from infpy.gp import SquaredExponentialKernel as SE
from infpy.gp import noise_kernel as noise

# close all existing plot windows
close('all')

#
# Part of X-space we are interested in
#
support = gp_1D_X_range(-10.0, 10.01, .125)

#
# Data
#
X = [[-5.], [-2.], [3.], [3.5]]
Y = [2.5, 2, -.5, 0.]

def plot_gp(gp, fig_title, filename):
  figure()
  plot([x[0] for x in X], Y, 'ks')
  mean, sigma, LL = gp.predict(support)
  gp_plot_prediction(support, mean, sigma)
  xlabel('x')
  ylabel('f(x)')
  title(fig_title)
  savefig('%s.png' % filename)
  savefig('%s.eps' % filename)
  
#
# Create a kernel with reasonable parameters and plot the GP predictions
#
kernel = SE([1.]) + noise(1.)
gp = GaussianProcess(X, Y, kernel)
plot_gp(
  gp=gp,
  fig_title='Initial parameters: kernel = SE([1]) + noise(1)',
  filename='learning_first_guess'
)

#
# Learn the covariance function's parameters and replot
#
gp_learn_hyperparameters(gp)
plot_gp(
  gp=gp,
  fig_title='Learnt parameters: kernel = SE([%.2f]) + noise(%.2f)' % (
    kernel.k1.params[0],
    kernel.k2.params.o2[0]
  ),
  filename='learning_learnt'
)
```

## 参考资料
[What are Gaussian processes?](https://pythonhosted.org/infpy/gps.html)